# 不同重連機制實驗結果分析

本筆記本用於分析和可視化不同重連機制（original、attention、spectral）的實驗結果，以便比較它們的性能。

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

# 設置繪圖風格
plt.style.use('ggplot')
sns.set_theme(style='whitegrid')

# 設置中文字體支持（如果需要）
# plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']  # 或其他支持中文的字體
# plt.rcParams['axes.unicode_minus'] = False  # 正確顯示負號

## 載入實驗結果數據

首先，我們需要找到並載入最新的實驗結果摘要文件。

In [2]:
# 找到最新的實驗結果摘要文件
results_dir = './results'
summary_files = glob.glob(f'{results_dir}/*_summary_*.csv')

if summary_files:
    # 按文件修改時間排序，獲取最新的文件
    latest_summary_file = max(summary_files, key=os.path.getmtime)
    print(f'找到最新的實驗結果摘要文件: {latest_summary_file}')
    
    # 讀取實驗結果摘要
    summary_df = pd.read_csv(latest_summary_file)
    display(summary_df)
else:
    print('未找到實驗結果摘要文件，請先運行實驗。')

找到最新的實驗結果摘要文件: ./results/Wisconsin_GCN_summary_20250401_024250.csv


,Rewiring,Train,Val,Test
0,original,0.909444,0.72250,0.664052
1,attention,NaN,NaN,NaN
2,spectral,NaN,NaN,NaN
3,baseline,0.640000,0.54375,0.496732


## 數據可視化

接下來，我們將實驗結果可視化，以便更直觀地比較不同重連機制的性能。

In [ ]:
# 繪製條形圖比較不同重連機制的性能
if 'summary_df' in locals():
    plt.figure(figsize=(12, 6))
    
    # 設置條形圖的位置
    x = np.arange(len(summary_df['Rewiring']))
    width = 0.25
    
    # 繪製三組條形圖（Train, Val, Test）
    plt.bar(x - width, summary_df['Train'], width, label='Train', color='#3498db')
    plt.bar(x, summary_df['Val'], width, label='Val', color='#2ecc71')
    plt.bar(x + width, summary_df['Test'], width, label='Test', color='#e74c3c')
    
    # 設置圖表標籤和標題
    plt.xlabel('重連機制', fontsize=12)
    plt.ylabel('準確率', fontsize=12)
    plt.title('不同重連機制的性能比較', fontsize=14)
    plt.xticks(x, summary_df['Rewiring'])
    plt.legend()
    
    # 在條形上方顯示數值
    for i, v in enumerate(summary_df['Test']):
        plt.text(i + width, v + 0.01, f'{v:.4f}', ha='center', fontsize=9)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 繪製折線圖比較不同重連機制的性能
if 'summary_df' in locals():
    plt.figure(figsize=(10, 6))
    
    # 繪製折線圖
    plt.plot(summary_df['Rewiring'], summary_df['Train'], 'o-', label='Train', linewidth=2, markersize=8)
    plt.plot(summary_df['Rewiring'], summary_df['Val'], 's-', label='Val', linewidth=2, markersize=8)
    plt.plot(summary_df['Rewiring'], summary_df['Test'], '^-', label='Test', linewidth=2, markersize=8)
    
    # 設置圖表標籤和標題
    plt.xlabel('重連機制', fontsize=12)
    plt.ylabel('準確率', fontsize=12)
    plt.title('不同重連機制的性能趨勢', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # 在數據點上顯示測試集準確率
    for i, v in enumerate(summary_df['Test']):
        plt.text(i, v + 0.01, f'{v:.4f}', ha='center', fontsize=9)
    
    plt.legend()
    plt.tight_layout()
    plt.show()

## 性能提升分析

計算各重連機制相對於基準（無重連）的性能提升百分比。

In [ ]:
# 計算性能提升百分比
if 'summary_df' in locals() and 'baseline' in summary_df['Rewiring'].values:
    # 獲取基準性能
    baseline_row = summary_df[summary_df['Rewiring'] == 'baseline']
    baseline_train = baseline_row['Train'].values[0]
    baseline_val = baseline_row['Val'].values[0]
    baseline_test = baseline_row['Test'].values[0]
    
    # 創建新的DataFrame來存儲性能提升
    improvement_df = summary_df[summary_df['Rewiring'] != 'baseline'].copy()
    improvement_df['Train_Improvement'] = ((improvement_df['Train'] - baseline_train) / baseline_train * 100).round(2)
    improvement_df['Val_Improvement'] = ((improvement_df['Val'] - baseline_val) / baseline_val * 100).round(2)
    improvement_df['Test_Improvement'] = ((improvement_df['Test'] - baseline_test) / baseline_test * 100).round(2)
    
    display(improvement_df)
    
    # 繪製性能提升條形圖
    plt.figure(figsize=(10, 6))
    
    x = np.arange(len(improvement_df['Rewiring']))
    width = 0.25
    
    plt.bar(x - width, improvement_df['Train_Improvement'], width, label='Train', color='#3498db')
    plt.bar(x, improvement_df['Val_Improvement'], width, label='Val', color='#2ecc71')
    plt.bar(x + width, improvement_df['Test_Improvement'], width, label='Test', color='#e74c3c')
    
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.xlabel('重連機制', fontsize=12)
    plt.ylabel('相對於基準的性能提升 (%)', fontsize=12)
    plt.title('不同重連機制相對於基準的性能提升', fontsize=14)
    plt.xticks(x, improvement_df['Rewiring'])
    
    # 在條形上方顯示測試集提升百分比
    for i, v in enumerate(improvement_df['Test_Improvement']):
        color = 'green' if v > 0 else 'red'
        plt.text(i + width, v + (1 if v > 0 else -3), f'{v:+.2f}%', ha='center', fontsize=9, color=color)
    
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print('未找到基準實驗結果，無法計算性能提升。')

## 結論

根據上述分析，我們可以得出以下結論：

1. **最佳重連機制**：根據測試集準確率，[填寫最佳重連機制]表現最好，達到了[填寫準確率]的準確率。

2. **性能提升**：與基準（無重連）相比，[填寫最佳重連機制]提高了[填寫提升百分比]%的測試集準確率。

3. **過擬合分析**：從訓練集和測試集的準確率差距來看，[填寫過擬合分析]。

4. **建議**：基於以上結果，建議在[填寫數據集名稱]數據集上使用[填寫建議的重連機制]重連機制與[填寫模型名稱]模型結合使用。

## 附錄：實驗日誌分析

如果需要更詳細地分析實驗日誌，可以使用以下代碼載入並分析日誌文件。

In [ ]:
# 找到並分析實驗日誌文件
def extract_epoch_data(log_file):
    epochs = []
    losses = []
    train_accs = []
    val_accs = []
    test_accs = []
    
    with open(log_file, 'r') as f:
        for line in f:
            if 'Epoch:' in line and 'Loss:' in line and 'Train:' in line:
                parts = line.strip().split(',')
                epoch = int(parts[0].split(':')[1].strip())
                loss = float(parts[1].split(':')[1].strip())
                train_acc = float(parts[2].split(':')[1].strip())
                val_acc = float(parts[3].split(':')[1].strip())
                test_acc = float(parts[4].split(':')[1].strip())
                
                epochs.append(epoch)
                losses.append(loss)
                train_accs.append(train_acc)
                val_accs.append(val_acc)
                test_accs.append(test_acc)
    
    return {
        'epochs': epochs,
        'losses': losses,
        'train_accs': train_accs,
        'val_accs': val_accs,
        'test_accs': test_accs
    }

# 找到最新的一組實驗日誌
log_files = {}
timestamp = None

for file in os.listdir(results_dir):
    if file.endswith('.log') and not file.endswith('_summary.log'):
        parts = file.split('_')
        if len(parts) >= 4:
            current_timestamp = parts[-1].replace('.log', '')
            if timestamp is None or current_timestamp > timestamp:
                timestamp = current_timestamp

if timestamp:
    for file in os.listdir(results_dir):
        if timestamp in file and file.endswith('.log') and not file.endswith('_summary.log'):
            if 'baseline' in file:
                log_files['baseline'] = os.path.join(results_dir, file)
            elif 'original' in file:
                log_files['original'] = os.path.join(results_dir, file)
            elif 'attention' in file:
                log_files['attention'] = os.path.join(results_dir, file)
            elif 'spectral' in file:
                log_files['spectral'] = os.path.join(results_dir, file)

    print(f'找到以下實驗日誌文件：')
    for key, file in log_files.items():
        print(f'- {key}: {file}')

    # 繪製訓練過程曲線
    plt.figure(figsize=(15, 10))
    
    # 損失曲線
    plt.subplot(2, 2, 1)
    for rewiring_type, log_file in log_files.items():
        data = extract_epoch_data(log_file)
        if data['epochs']:
            plt.plot(data['epochs'], data['losses'], label=rewiring_type)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # 訓練集準確率曲線
    plt.subplot(2, 2, 2)
    for rewiring_type, log_file in log_files.items():
        data = extract_epoch_data(log_file)
        if data['epochs']:
            plt.plot(data['epochs'], data['train_accs'], label=rewiring_type)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # 驗證集準確率曲線
    plt.subplot(2, 2, 3)
    for rewiring_type, log_file in log_files.items():
        data = extract_epoch_data(log_file)
        if data['epochs']:
            plt.plot(data['epochs'], data['val_accs'], label=rewiring_type)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Validation Accuracy')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # 測試集準確率曲線
    plt.subplot(2, 2, 4)
    for rewiring_type, log_file in log_files.items():
        data = extract_epoch_data(log_file)
        if data['epochs']:
            plt.plot(data['epochs'], data['test_accs'], label=rewiring_type)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Test Accuracy')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.show()
else:
    print('未找到實驗日誌文件。')